In [1]:
import sys
sys.path.append("../")

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import requests
import json

from pprint import pprint as print_
from collections import OrderedDict
import numpy as np
from tqdm import tqdm_notebook as tqdm

from isanlp_srl_framebank.pipeline_default import PipelineDefault

In [4]:
IAM_TOKEN = 't1.9euelZqPzpeZkI-WmM6RjIucmcmTyO3rnpWaj46Nkc6ejZaLyZvOmZqWj53l9PdNfXVk-e8KGD_O3fT3DSxzZPnvChg_zg.x-M9U8Er7_xZaZ5_Wyt3Aa5wpmi4K7Ln1pNnL_ZOaTM9dBJHc0ejbeXwJ3L0Rjwqlly5jeOfRiqmIKMwU7EZDg'
folder_id = 'b1gs2kplab3ve997i80t'

In [ ]:
# docker run --rm -p 3333:3333 inemo/isanlp
# docker run --rm --shm-size=1024m -ti -p 3334:9999 inemo/syntaxnet_rus server 0.0.0.0 9999
# docker run --rm -p 3335:3333 inemo/isanlp_srl_framebank

In [5]:
ppl = PipelineDefault(address_morph=('localhost', 3333),
                      address_syntax=('localhost', 3334),
                      address_srl=('localhost', 3335))

In [6]:
def print_roles(lemma, role_annot, translate=False):
    for sent_num, ann_sent in enumerate(role_annot):
        for event in ann_sent:
            if translate:
                print('=====Pred: {}'.format(*get_translations(lemma[sent_num][event.pred[0]],'tt')))
            else:
                print('=====Pred: {}'.format(lemma[sent_num][event.pred[0]]))
            for arg in event.args:
                if translate:
                    print('Arg({}): {}'.format(arg.tag, *get_translations(lemma[sent_num][arg.begin], 'tt')))
                else:
                    print('Arg({}): {}'.format(arg.tag, lemma[sent_num][arg.begin]))

In [7]:
def get_translations(texts, target_language='ru' ):

    body = {
        "targetLanguageCode": target_language,
        "texts": texts,
        "folderId": folder_id,
    }

    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer {0}".format(IAM_TOKEN)
    }

    response = requests.post('https://translate.api.cloud.yandex.net/translate/v2/translate',
        json = body,
        headers = headers
    )
    d = json.loads(response.text)
    translations = d['translations']
    return [t['text'] for t in translations]
    
# print(response.text)

In [9]:
texts = ["Без дачага киттек.", "Балалар өйдән кача һәм урманда югалып кала.","Әни рамны юды. Әти машинаны йөртте."]
translations = get_translations(texts)
# print(translations)
for i, translation in enumerate(translations):
    print('___исходный текст на татарском____')
    print(texts[i])
    rus_text = translation
    print('___переведенный текст на русский____')
    print(rus_text)
    res = ppl(rus_text)
    print('______На русском_____')
    print_roles(res['lemma'], res['srl'])
    print('______На татарском____')
    print_roles(res['lemma'], res['srl'], True)


    

___исходный текст на татарском____
Без дачага киттек.
___переведенный текст на русский____
Мы поехали на дачу.
______На русском_____
=====Pred: поехать
Arg(субъект перемещения): мы
Arg(конечная точка): дача
______На татарском____
=====Pred: бару
Arg(субъект перемещения): без
Arg(конечная точка): дача
___исходный текст на татарском____
Балалар өйдән кача һәм урманда югалып кала.
___переведенный текст на русский____
Дети убегают из дома и теряются в лесу.
______На русском_____
=====Pred: убегать
Arg(субъект перемещения): ребенок
Arg(начальная точка): дом
=====Pred: теряться
Arg(место): лес
______На татарском____
=====Pred: качу
Arg(субъект перемещения): балалар
Arg(начальная точка): йорт
=====Pred: югалту
Arg(место): урман
___исходный текст на татарском____
Әни рамны юды. Әти машинаны йөртте.
___переведенный текст на русский____
Мама вымыла раму. Отец водил машину.
______На русском_____
=====Pred: вымывать
Arg(агенс): мама
Arg(пациенс): рама
=====Pred: водить
Arg(агенс): отец
______На та